# TubeTalk Test cases  

This notebook represents the initial version of the project. If you’d like to test the code without using the Chainlit interface, you can run this notebook locally and enter a YouTube link as input. 

In the output sections, you’ll find the results based on our test and we performed it on a short video.

## Import Required Libraries

In [ ]:

import os
import shutil
import uuid
import tempfile
import asyncio
from io import BytesIO
import whisper
import yt_dlp
import re 
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool
import chainlit as cl
from chainlit.element import Element
import numpy as np
import soundfile as sf 

## YouTube Audio Download and Transcription

In [8]:
import yt_dlp
import os

# Download and convert YouTube audio to mp3
def download_audio(youtube_url, filename="./audio.mp3"):
    print("Starting download...")
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': filename,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
    
    if os.path.exists(filename):
        print(f"Downloaded succesfully: {filename}")
    else:
        print(f"Downloaded failed: {filename}")
    return filename

In [ ]:
import whisper

# Convert mp3 to wav
def transcribe_audio(audio_path):
    print("Starting transcription...")
    model = whisper.load_model("base") 
    result = model.transcribe(audio_path)
    text = result['text']
    print("Transcription completed.")

    # Create a text file with the transcribed text
    text_filename = "transcribed_text.txt"

    # Open the file in write mode and save the transcribed text
    with open(text_filename, "w", encoding="utf-8") as file:
        file.write(text)

    print(f"Transcribed text has been saved successfully to {text_filename}.")
    
    return text


In [ ]:
# Testing the functions
youtube_url = input("Enter the youtube URL: ").strip()

audio_file = download_audio(youtube_url)

if os.path.exists(audio_file):
    text = transcribe_audio(audio_file)
else:
    print("Failed to download audio file.")


Starting download...
Downloaded succesfully: ./audio.mp3
Starting transcription...


c:\Users\Nada\Documents\AI Engineering bootcamp\final project\myenv\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription completed.

Transcribed Text:
 In this video, we're going to be talking about the promptub. The promptub is a feature in Lensmith that allows us to save and version prompts that we've been iterating on and refining. Once we've saved our prompt, we can then also pull it locally into our application and reuse it in our code. To show you how to do this, we're going to continue with our example with our parent named Poly. As a reminder, Poly is a parent and Poly has some facts that she can use to answer user's questions about he


## Tokenization and Chunking

In [2]:

# Reading the transcribed text from the file
text_filename = "transcribed_text.txt"
with open(text_filename, "r", encoding="utf-8") as file:
    loaded_text = file.read()

print(f"Transcribed text has been loaded successfully from {text_filename}.")


Transcribed text has been loaded successfully from transcribed_text.txt.


In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Tokenize and chunk the text using HuggingFace tokenizer
def tokenize_and_chunk_hf(text, max_tokens=256):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return [tokenizer.decode(tokens[i:i + max_tokens]).strip() for i in range(0, len(tokens), max_tokens)]



In [ ]:
# Testing the chunking function
chunks = tokenize_and_chunk_hf(loaded_text)

# Display the chunks
print(f"Total chunks: {len(chunks)}")
print("All chunks preview:")
print("="*50)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print(chunk)
    print("="*50)

Text has been smartly split into 2 chunks successfully.
Total smart chunks: 2
All chunks preview:
Chunk 1:
In this video, we're going to be talking about the promptub. The promptub is a feature in Lensmith that allows us to save and version prompts that we've been iterating on and refining. Once we've saved our prompt, we can then also pull it locally into our application and reuse it in our code. To show you how to do this, we're going to continue with our example with our parent named Poly. As a reminder, Poly is a parent and Poly has some facts that she can use to answer user's questions about herself. Let's pretend that we've iterated on this prompt for some time now and we're pretty happy with it and ready to save it for reuse. It's probably not a good idea to save this prompt with these inputs hard-coded in and also with this question from the user hard-coded. What we can do is we can replace these facts with an input variable. This essentially allows the user to, at runtime, pas

## Build Vectorstore

In [ ]:
# Create embeddings for each chunk and store them in ChromaDB

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os

# Initialize a directory to store the database
persist_directory = "chroma_db"

# Initialize the text-to-embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a new Chroma database or load an existing one
# Check if the directory exists, if not create it
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)


C:\Users\Nada\AppData\Local\Temp\ipykernel_33496\2699622044.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Nada\Documents\AI Engineering bootcamp\final project\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Create a list of documents (chunks) to be stored in the database
documents = chunks 

# Create the Chroma vector store from the documents and embeddings
vectorstore = Chroma.from_texts(documents, embedding_model, persist_directory=persist_directory)

# Save the vector store to the specified directory
vectorstore.persist()


C:\Users\Nada\AppData\Local\Temp\ipykernel_33496\1287538581.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [19]:
print(f"Embeddings have been generated and stored successfully in '{persist_directory}' directory.")
print(f"Total chunks stored: {len(documents)}")

Embeddings have been generated and stored successfully in 'chroma_db' directory.
Total chunks stored: 2


## RAG

In [20]:
# Searching for the most relevant chunk that answers the user's question using ChromaDB
from langchain.vectorstores import Chroma

# Download the database from the specified directory
persist_directory = "chroma_db"
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

# Function to search for similar chunks in the database
def search_similar_chunks(question, top_k=2):
    """
    Search for similar chunks in the database using Embedding
    Args:
        question (str): The question to search for
        top_k (int): The number of best results to retrieve
    Returns:
        list: A list of relevant chunks
    """
    # Perform similarity search in the database
    results = vectorstore.similarity_search(question, k=top_k)
    
    if results:
        print(f"Found {len(results)} relevant chunk(s).")
        return [doc.page_content for doc in results]
    else:
        print("No relevant information found for the question.")
        return []


C:\Users\Nada\AppData\Local\Temp\ipykernel_33496\2225495744.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)


In [ ]:
# This part gives us the closest chunk only but does not answer the question
# Testing the search function
# Taking a question from the user
user_question = input("Enter your question about the video: ").strip()

# Applying the search function to find relevant chunks
relevant_chunks = search_similar_chunks(user_question, top_k=2)

# Display the relevant chunks
if relevant_chunks:
    print("\nRelevant Chunk(s) Retrieved:")
    print("="*50)
    for idx, chunk in enumerate(relevant_chunks):
        print(f"Chunk {idx+1}:\n{chunk}")
        print("="*50)
else:
    print("No relevant information could be retrieved.")


Found 2 relevant chunk(s).

Relevant Chunk(s) Retrieved:
Chunk 1:
In this video, we're going to be talking about the promptub. The promptub is a feature in Lensmith that allows us to save and version prompts that we've been iterating on and refining. Once we've saved our prompt, we can then also pull it locally into our application and reuse it in our code. To show you how to do this, we're going to continue with our example with our parent named Poly. As a reminder, Poly is a parent and Poly has some facts that she can use to answer user's questions about herself. Let's pretend that we've iterated on this prompt for some time now and we're pretty happy with it and ready to save it for reuse. It's probably not a good idea to save this prompt with these inputs hard-coded in and also with this question from the user hard-coded. What we can do is we can replace these facts with an input variable. This essentially allows the user to, at runtime, pass in the values that they want to use for

## Load API Keys

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

In [12]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [13]:
# Initialize the OpenAI model
# LangChain 
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-4",  
    temperature=0.2
)

C:\Users\Nada\AppData\Local\Temp\ipykernel_33496\2744230690.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [ ]:
# Here takes the question and searches for the best chunk and answers it
# Retrieval-Augmented Generation (RAG) pipeline
# This function will search for the most relevant chunks based on the user's question
def rag_qa_pipeline(user_question, top_k=2):
    """
    Applying the smart search and then generating an answer based on the retrieved texts
    Args:
        user_question (str): user's question
        top_k (int): number of best result
    Returns:
        str: smart answer to the user's question
    """
    # Searching for the most relevant chunks that answer the user's question
    relevant_chunks = search_similar_chunks(user_question, top_k=top_k)

    if not relevant_chunks:
        return "Sorry, I couldn't find relevant information to answer your question."

    # Prepare the context for the model (Augmentation)
    context_text = "\n\n".join(relevant_chunks)
    prompt_template = """
    You are an expert assistant. Use the following context to answer the question.
    Make sure your answer is in the same language as the question.

    Context:
    {context}
    
    Question:
    {question}
    
    Answer in a clear, complete, and concise way:
    """

    final_prompt = PromptTemplate.from_template(prompt_template)
    chain = LLMChain(llm=llm, prompt=final_prompt)

    # Generate the answer using the LLM
    answer = chain.run(context=context_text, question=user_question)

    return answer

In [ ]:
# Taking a question from the user
user_question = input("Enter your question about the video: ").strip()

# Execute the full RAG pipeline
final_answer = rag_qa_pipeline(user_question)

# Display the final answer
print("\nThe question was:")
print(user_question)
print("\nThe Answer:")
print("="*50)
print(final_answer)
print("="*50)


Found 2 relevant chunk(s).


C:\Users\Nada\AppData\Local\Temp\ipykernel_14376\3830563266.py:33: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=final_prompt)
C:\Users\Nada\AppData\Local\Temp\ipykernel_14376\3830563266.py:36: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = chain.run(context=context_text, question=user_question)



Final Answer:
The video is discussing the use of the 'promptub' feature in Lensmith, a tool that allows users to save, version, and reuse prompts that have been refined over time. The tutorial uses an example of a character named Poly to demonstrate how to replace hard-coded inputs with variables for user customization. The video also shows how to save the prompt for private or public use, and how to pull the prompt into the user's code. The tutorial further explains how to make revisions to the saved prompt, such as changing the language of the response, and how these changes are tracked in the commit history.


## RAGAS

In [ ]:
# The whole process: Question ➔ Search ➔ Generate ➔ Direct Evaluation
# A list of questions for testing (e.g., the questions we defined earlier)
questions = [
    "How can users modify hard-coded inputs in a saved prompt?",
    "What happens when you save a prompt privately or publicly in the prompt hub?",
    "How does the system behave if you edit a prompt and commit a new version?",
    "What effect does not specifying a commit hash have when using a prompt?"
]
reference_answers = [
    "Users can modify hard-coded inputs in a saved prompt by replacing these facts with an input variable. This allows the user to pass in the values they want to use for both the facts and the question at runtime. This can be done in the Lensmith's promptub feature where the prompt is saved and versioned. After making these changes, users can save the prompt for reuse.",
    "Saving a prompt privately or publicly lets users reuse and version prompts via the prompt hub.",
    "Editing a prompt and committing a new version updates the prompt in the commit history for version control.",
    "Not specifying a commit hash always pulls the latest version of the prompt."
]
retrieved_contexts_all = []
generated_answers_all = []

# Retrieval and generation for each question
for question in questions:
    # Search for the most relevant chunks that answer the user's question
    retrieved_chunks = search_similar_chunks(question, top_k=3)
    
    # Generate the answer using the RAG pipeline
    answer = rag_qa_pipeline(question)
    
    # Save the retrieved chunks and generated answer for each question
    retrieved_contexts_all.append(retrieved_chunks)
    generated_answers_all.append(answer)

print("\nAll questions answered and contexts retrieved.")


Found 3 relevant chunk(s).
Found 2 relevant chunk(s).
Found 3 relevant chunk(s).
Found 2 relevant chunk(s).
Found 3 relevant chunk(s).
Found 2 relevant chunk(s).
Found 3 relevant chunk(s).
Found 2 relevant chunk(s).

All questions answered and contexts retrieved.


### Evaluation 

In [ ]:
# Evaluating the generated answers using RAGAS
import pandas as pd
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision
from datasets import Dataset

# Prepare the data for evaluation
evaluation_dataset = pd.DataFrame({
    "question": questions,
    "answer": generated_answers_all,
    "contexts": retrieved_contexts_all,
    "reference": reference_answers 
})

# Convert the DataFrame to a Dataset object for RAGAS evaluation
evaluation_dataset = Dataset.from_pandas(evaluation_dataset)

# Evaluate the generated answers using RAGAS
evaluation_result = evaluate(
    evaluation_dataset,
    metrics=[
        faithfulness,        
        answer_relevancy,    
        context_precision      
    ]
)


Evaluating: 100%|██████████| 12/12 [00:14<00:00,  1.23s/it]


In [ ]:
# Display the evaluation results
print("\nFinal Evaluation Results:")
print("="*50)
print(evaluation_result)
print("="*50)



Final Evaluation Results:
{'faithfulness': 1.0000, 'answer_relevancy': 0.9749, 'context_precision': 1.0000}
